### Import Required Libraries and Set Up Environment Variables

In [83]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import os
from datetime import datetime
## Load the NASA_API_KEY from the env file
load_dotenv()
NASA_API_KEY = os.getenv('NASA_API_KEY')

### CME Data

In [84]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for CMEs:
CME = "CME"

# Search for CMEs published between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for CME
query_url_CME =  f"{base_url}{CME}?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"


In [86]:
# Make a "GET" request for the CME URL and store it in a variable named cme_response
cme_response = requests.get(query_url_CME)

In [87]:
# Cell to redo when ready Convert the response variable to json and store it as a variable named cme_json
cme_json = cme_response.json()

In [88]:
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(cme_json[:1], indent=4))

[
    {
        "activityID": "2013-05-01T03:12:00-CME-001",
        "catalog": "M2M_CATALOG",
        "startTime": "2013-05-01T03:12Z",
        "instruments": [
            {
                "displayName": "SOHO: LASCO/C2"
            },
            {
                "displayName": "SOHO: LASCO/C3"
            },
            {
                "displayName": "STEREO A: SECCHI/COR2"
            },
            {
                "displayName": "STEREO B: SECCHI/COR2"
            }
        ],
        "sourceLocation": "",
        "activeRegionNum": null,
        "note": "",
        "submissionTime": "2013-08-07T16:54Z",
        "versionId": 1,
        "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/CME/2349/-1",
        "cmeAnalyses": [
            {
                "isMostAccurate": true,
                "time21_5": "2013-05-01T07:07Z",
                "latitude": 12.0,
                "longitude": -120.0,
                "halfAngle": 36.0,
                "speed": 860.0,
        

In [89]:
# Convert cme_json to a Pandas DataFrame 
# Keep only the columns: activityID, startTime, linkedEvents
cme_df = pd.DataFrame(cme_json, columns=["activityID", "startTime", "linkedEvents"])
cme_df.head()

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
1,2013-05-02T05:24:00-CME-001,2013-05-02T05:24Z,None
2,2013-05-02T14:36:00-CME-001,2013-05-02T14:36Z,None
3,2013-05-03T18:00:00-CME-001,2013-05-03T18:00Z,None
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]


In [90]:
# Notice that the linkedEvents column allows us to identify the corresponding GST
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs
cme_clean_df = cme_df.dropna()
cme_clean_df.head()

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]
7,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,[{'activityID': '2013-05-12T23:30:00-IPS-001'}]
10,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,[{'activityID': '2013-05-13T01:53:00-FLR-001'}...
13,2013-05-13T16:18:00-CME-001,2013-05-13T16:18Z,[{'activityID': '2013-05-13T15:40:00-FLR-001'}...


In [91]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Write a nested for loop that iterates first over each row in the cme DataFrame (using the index)
# and then iterates over the values in 'linkedEvents' 
# and adds the elements individually to a list of dictionaries where each row is one element 

# Initialize an empty list to store the expanded rows
cme_list = []

# Iterate over each index in the DataFrame
for each_index in cme_clean_df.index:

    # Iterate over each dictionary in the list
    linked_events = cme_clean_df.loc[each_index, 'linkedEvents']
    for event in linked_events:
        event_dict = {
                'activityID': cme_clean_df.loc[each_index, 'activityID'],
                'startTime': cme_clean_df.loc[each_index, 'startTime'],
                'linkedEvent': event
        }
    
        # Append a new dictionary to the expanded_rows list for each dictionary item and corresponding 'activityID' and 'startTime' value
        cme_list.append(event_dict)
      
# Create a new DataFrame from the expanded rows
expanded_cme_df = pd.DataFrame(cme_list)

expanded_cme_df.head()


,activityID,startTime,linkedEvent
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'}
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'}
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'}
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'}
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'}


In [92]:
# Create a function called extract_activityID_from_dict that takes a dict as input such as in linkedEvents
# and verify below that it works as expected using one row from linkedEvents as an example
# Be sure to use a try and except block to handle errors
def extract_activityID_from_dict(event_dict):
    try:
        return event_dict['activityID']
    except KeyError as e:
        print(f"KeyError: {e} - The key 'activityID' key was not found.")
        return None
first_row_dict = expanded_cme_df['linkedEvent'].iloc[0]
activity_id = extract_activityID_from_dict(first_row_dict)

        # Log the error or print it for debugging
print(activity_id)



2013-05-04T04:52:00-IPS-001


In [93]:
# Apply this function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'GST_ActivityID' using loc indexer:
expanded_cme_df['GST_ActivityID'] = expanded_cme_df['linkedEvent'].apply(lambda x: extract_activityID_from_dict(x))
expanded_cme_df.head()


,activityID,startTime,linkedEvent,GST_ActivityID
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'},2013-05-04T04:52:00-IPS-001
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'},2013-05-07T04:37:00-IPS-001
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'},2013-05-12T23:30:00-IPS-001
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'},2013-05-13T01:53:00-FLR-001
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'},2013-05-13T04:12:00-SEP-001


In [94]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs:
expanded_cme_df = expanded_cme_df.dropna()
expanded_cme_df.head()

,activityID,startTime,linkedEvent,GST_ActivityID
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,{'activityID': '2013-05-04T04:52:00-IPS-001'},2013-05-04T04:52:00-IPS-001
1,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,{'activityID': '2013-05-07T04:37:00-IPS-001'},2013-05-07T04:37:00-IPS-001
2,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,{'activityID': '2013-05-12T23:30:00-IPS-001'},2013-05-12T23:30:00-IPS-001
3,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T01:53:00-FLR-001'},2013-05-13T01:53:00-FLR-001
4,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,{'activityID': '2013-05-13T04:12:00-SEP-001'},2013-05-13T04:12:00-SEP-001


In [95]:
# print out the datatype of each column in this DataFrame:
expanded_cme_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1714 entries, 0 to 1713
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   activityID      1714 non-null   object
 1   startTime       1714 non-null   object
 2   linkedEvent     1714 non-null   object
 3   GST_ActivityID  1714 non-null   object
dtypes: object(4)
memory usage: 53.7+ KB


In [96]:
# Convert the 'GST_ActivityID' column to string format
expanded_cme_df['GST_ActivityID'] = pd.Series(expanded_cme_df['GST_ActivityID'], dtype='string') # Also didn't work, 'astype(str)' and 'apply(str)'

# Convert startTime to datetime format
expanded_cme_df['startTime'] = pd.to_datetime(expanded_cme_df['startTime'])

# Rename startTime to startTime_CME and activityID to cmeID
expanded_cme_df = expanded_cme_df.rename(columns={
    'startTime': 'startTime_CME', 
    'activityID': 'cmeID'
    })

# Drop linkedEvents
expanded_cme_df = expanded_cme_df.drop(columns='linkedEvent')

# Verify that all steps were executed correctly
expanded_cme_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1714 entries, 0 to 1713
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   cmeID           1714 non-null   object             
 1   startTime_CME   1714 non-null   datetime64[ns, UTC]
 2   GST_ActivityID  1714 non-null   string             
dtypes: datetime64[ns, UTC](1), object(1), string(1)
memory usage: 40.3+ KB


In [97]:
# We are only interested in CMEs related to GSTs so keep only rows where the GST_ActivityID column contains 'GST'
# use the method 'contains()' from the str library.  
gst_cme_df = expanded_cme_df.loc[expanded_cme_df['GST_ActivityID'].str.contains('GST')]
cme_df_merge = gst_cme_df
cme_df_merge.info()


<class 'pandas.core.frame.DataFrame'>
Index: 61 entries, 21 to 1689
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   cmeID           61 non-null     object             
 1   startTime_CME   61 non-null     datetime64[ns, UTC]
 2   GST_ActivityID  61 non-null     string             
dtypes: datetime64[ns, UTC](1), object(1), string(1)
memory usage: 1.9+ KB


### GST Data

In [98]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for Geomagnetic Storms (GST):
GST = "GST"

# Search for GSTs between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for GST
query_url_GST = f'{base_url}{GST}?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}'

In [99]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response
gst_response = requests.get(query_url_GST)

In [100]:
# Convert the response variable to json and store it as a variable named gst_json
gst_json = gst_response.json()

# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(gst_json, indent=4))


[
    {
        "gstID": "2013-06-01T01:00:00-GST-001",
        "startTime": "2013-06-01T01:00Z",
        "allKpIndex": [
            {
                "observedTime": "2013-06-01T01:00Z",
                "kpIndex": 6.0,
                "source": "NOAA"
            }
        ],
        "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/326/-1",
        "linkedEvents": [
            {
                "activityID": "2013-05-31T15:45:00-HSS-001"
            }
        ],
        "submissionTime": "2013-07-15T19:26Z",
        "versionId": 1
    },
    {
        "gstID": "2013-06-07T03:00:00-GST-001",
        "startTime": "2013-06-07T03:00Z",
        "allKpIndex": [
            {
                "observedTime": "2013-06-07T03:00Z",
                "kpIndex": 6.0,
                "source": "NOAA"
            }
        ],
        "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/330/-1",
        "linkedEvents": [
            {
                "activityID": "2013-06-02T20:24:

In [101]:
# Convert gst_json to a Pandas DataFrame
gst_json_df = pd.DataFrame(gst_json)

# Keep only the columns: gstID, startTime, linkedEvents
gst_json_df_ltd = gst_json_df[['gstID', 'startTime', 'linkedEvents']]
gst_json_df_ltd

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
2,2013-06-29T03:00:00-GST-001,2013-06-29T03:00Z,None
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...
...,...,...,...
112,2023-12-18T06:00:00-GST-001,2023-12-18T06:00Z,[{'activityID': '2023-12-14T17:38:00-CME-001'}...
113,2024-03-03T18:00:00-GST-001,2024-03-03T18:00Z,[{'activityID': '2024-02-28T17:48:00-CME-001'}...
114,2024-03-23T21:00:00-GST-001,2024-03-23T21:00Z,[{'activityID': '2024-03-23T12:49:00-IPS-001'}...
115,2024-03-24T12:00:00-GST-001,2024-03-24T12:00Z,[{'activityID': '2024-03-23T01:25:00-CME-001'}...


In [102]:
# Notice that the linkedEvents column allows us to identify the corresponding CME
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to CME
gst_json_df_ltd_clean = gst_json_df_ltd.dropna()
gst_json_df_ltd_clean.head()

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...
5,2014-02-19T03:00:00-GST-001,2014-02-19T03:00Z,[{'activityID': '2014-02-16T14:15:00-CME-001'}...


In [103]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Use the explode method to ensure that each row is one element. Ensure to reset the index and drop missing values.

# Explode to achieve one element per row
gst_df_explode = gst_json_df_ltd_clean.explode('linkedEvents')

# Reset the index
gst_df_explode.reset_index(drop=True, inplace=True)

# Drop missing values
gst_df_explode.dropna(subset=['linkedEvents'], inplace=True)

gst_df_explode.head()

,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,{'activityID': '2013-05-31T15:45:00-HSS-001'}
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,{'activityID': '2013-06-02T20:24:00-CME-001'}
2,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-09-29T22:40:00-CME-001'}
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T01:54:00-IPS-001'}
4,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T02:47:00-MPC-001'}


In [104]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'CME_ActivityID' using loc indexer:
gst_df_explode['CME_ActivityID'] = gst_df_explode['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x))

# Remove rows with missing CME_ActivityID, since we can't assign them to CMEs:
gst_df_explode.dropna(subset=['CME_ActivityID'], inplace=True)
gst_df_explode.head()


,gstID,startTime,linkedEvents,CME_ActivityID
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,{'activityID': '2013-05-31T15:45:00-HSS-001'},2013-05-31T15:45:00-HSS-001
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,{'activityID': '2013-06-02T20:24:00-CME-001'},2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-09-29T22:40:00-CME-001'},2013-09-29T22:40:00-CME-001
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T01:54:00-IPS-001'},2013-10-02T01:54:00-IPS-001
4,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T02:47:00-MPC-001'},2013-10-02T02:47:00-MPC-001


In [105]:
gst_df_explode.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   gstID           205 non-null    object
 1   startTime       205 non-null    object
 2   linkedEvents    205 non-null    object
 3   CME_ActivityID  205 non-null    object
dtypes: object(4)
memory usage: 6.5+ KB


In [106]:
# Convert the 'CME_ActivityID' column to string format
#gst_df_explode['CME_ActivityID'] = pd.Series(gst_df_explode['CME_ActivityID'], dtype='str') # Also didn't work, 'astype(str)' and 'apply(str)'
gst_df_explode['CME_ActivityID'] = pd.Series(gst_df_explode['CME_ActivityID'], dtype='string')

# Convert the 'gstID' column to string format
gst_df_explode['gstID'] = pd.Series(gst_df_explode['gstID'], dtype='string') # Also didn't work, 'astype(str)' and 'apply(str)'

# Convert startTime to datetime format  
gst_df_explode['startTime'] = pd.to_datetime(gst_df_explode['startTime'])

# Rename startTime to startTime_GST
#gst_df_explode = gst_df_explode.rename(columns={'startTime': 'startTime_GST'})
gst_df_explode.rename(columns={'startTime':'startTime_GST'}, inplace=True)

# Drop linkedEvents
#gst_df_explode = gst_df_explode.drop('linkedEvents', axis=1)
gst_df_explode.drop(columns=['linkedEvents'], inplace=True)

# Verify that all steps were executed correctly
gst_df_explode.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   gstID           205 non-null    string             
 1   startTime_GST   205 non-null    datetime64[ns, UTC]
 2   CME_ActivityID  205 non-null    string             
dtypes: datetime64[ns, UTC](1), string(2)
memory usage: 4.9 KB


In [107]:
# We are only interested in GSTs related to CMEs so keep only rows where the CME_ActivityID column contains 'CME'
# use the method 'contains()' from the str library.
gst_df_cme_only = gst_df_explode.loc[gst_df_explode['CME_ActivityID'].str.contains('CME')]
gst_df_merge = gst_df_cme_only
gst_df_merge.head()

,gstID,startTime_GST,CME_ActivityID
1,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001
5,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001
7,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001
9,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001


### Merge both datatsets

In [108]:
# Now merge both datasets
# GST = gst_df_merge ('gstID' and 'CME_ActivityID')
# CME = cme_df_merge ('cmeID' and 'GST_ActivityID')
# Use the 'left_on' and 'right_on' specifiers.

merged_df = pd.merge(gst_df_merge, cme_df_merge, 
                     left_on=['gstID','CME_ActivityID'], 
                     right_on=['GST_ActivityID','cmeID']
                     )
merged_df.head()


,gstID,startTime_GST,CME_ActivityID,cmeID,startTime_CME,GST_ActivityID
0,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001,2013-06-02T20:24:00-CME-001,2013-06-02 20:24:00+00:00,2013-06-07T03:00:00-GST-001
1,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001,2013-09-29T22:40:00-CME-001,2013-09-29 22:40:00+00:00,2013-10-02T03:00:00-GST-001
2,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001,2013-12-04T23:12:00-CME-001,2013-12-04 23:12:00+00:00,2013-12-08T00:00:00-GST-001
3,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001,2014-02-16T14:15:00-CME-001,2014-02-16 14:15:00+00:00,2014-02-19T03:00:00-GST-001
4,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001,2014-02-18T01:25:00-CME-001,2014-02-18 01:25:00+00:00,2014-02-20T03:00:00-GST-001


In [109]:
# Verify that the new DataFrame has the same number of rows as cme (61) and gst (61)
merged_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   gstID           61 non-null     string             
 1   startTime_GST   61 non-null     datetime64[ns, UTC]
 2   CME_ActivityID  61 non-null     object             
 3   cmeID           61 non-null     object             
 4   startTime_CME   61 non-null     datetime64[ns, UTC]
 5   GST_ActivityID  61 non-null     string             
dtypes: datetime64[ns, UTC](2), object(2), string(2)
memory usage: 3.0+ KB


### Computing the time it takes for a CME to cause a GST

In [110]:
# Compute the time diff between startTime_GST and startTime_CME by creating a new column called `timeDiff`.
merged_df['timeDiff'] = merged_df['startTime_GST'] - merged_df['startTime_CME']

In [111]:
# Use describe() to compute the mean and median time 
# that it takes for a CME to cause a GST. 
merged_df.describe()

,timeDiff
count,61
mean,2 days 23:29:26.557377049
std,0 days 23:53:09.336914240
min,1 days 08:36:00
25%,2 days 04:00:00
50%,2 days 19:51:00
75%,3 days 13:35:00
max,6 days 03:00:00


### Exporting data in csv format

In [112]:
# Export data to CSV without the index
merged_df.to_csv('output-Reference/merged_output_file.csv', index=False)